In [1]:
from binance import Client
client = Client(api_key="VQaW84v81fvM6b0z2cG97q0FBGK0VDVNYZGxPlMYkL1caao8u7lSrSWxeE5c6NKq",api_secret="9XgnO5nds9V8UVw5O8D0J6NHNPqL4ojj5t3FxZs7xqc8sd8MwEt44QshkT3W9auM",tld='com')

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd
from time import sleep

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    # print('Indicadores agregados')

def señal_MACD(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0:
            buy.append(i)
        elif df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0:
            sell.append(i)
    return buy, sell

In [2]:
from datetime import datetime,timedelta

day = datetime.utcnow() - timedelta(1)
PAIR = 'BTCTUSD'
LOCALIDAD_SEÑAL_ENTRADA = 10
x = 0
y = 0

while True:
    with connection.connect() as conn:
        df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
    df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
    df = df.sort_values(by='Tiempo')
    MACD(df)
    buy, sell = señal_MACD(df)

    buy = df.Precio.iloc[buy]
    sell = df.Precio.iloc[sell]
    DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buy.index[-1]).total_seconds()
    DIFF_SEÑAL_SALIDA = (datetime.utcnow() - sell.index[-1]).total_seconds()
    if x != buy.index[-1]:
        print('entrada', DIFF_SEÑAL_ENTRADA)
        x=buy.index[-1]
    if y != sell.index[-1]:
        print('salida', DIFF_SEÑAL_SALIDA)
        y=sell.index[-1]

entrada 18.292702
salida 62.661702
salida 5.956832
entrada 7.28125
salida 3.29425
entrada 4.556352
salida 3.851562
entrada 4.948492
salida 9.028322
entrada 6.2745429999999995
salida 4.829607
entrada 8.749692
entrada 33.235599
salida 67.404599
entrada 10.622012
salida 13.415012
entrada 3.623996
salida 6.451996
entrada 7.191167
salida 3.918167
entrada 6.827418
entrada 4.715549
salida 3.601549
entrada 7.930704
salida 7.801636
entrada 4.597713
entrada 6.225301
salida 19.229301
entrada 24.520565
salida 3.123565
entrada 22.692136
salida 20.912136
entrada 14.365193
salida 33.478193
entrada 26.471822
salida 9.613822
entrada 30.174725
salida 8.267725
entrada 5.599736
salida 5.633301
entrada 4.451172
entrada 3.530155
salida 2.708155
entrada 8.103428
salida 3.976428
entrada 7.060095
salida 6.555748
entrada 7.923058
salida 5.900058
entrada 8.013418
salida 7.324208
entrada 6.964337
salida 3.455748
entrada 7.508432
salida 6.817432
entrada 4.503929
salida 7.286774
entrada 5.938744
salida 4.816744
ent

In [123]:
buy.index[-1]

Timestamp('2023-07-05 00:31:53.811000')

In [3]:
import socket
print(socket.gethostbyname(socket.gethostname()))

192.168.0.215


In [10]:
client.get_isolated_margin_account(symbols='BTCTUSD')['assets'][0]['quoteAsset']['free']

'43.9163194'

In [6]:
client.get_account_api_permissions()

{'ipRestrict': True,
 'createTime': 1688443776000,
 'enableReading': True,
 'enableSpotAndMarginTrading': True,
 'enableWithdrawals': False,
 'enableInternalTransfer': False,
 'enableMargin': True,
 'enableFutures': True,
 'permitsUniversalTransfer': True,
 'enableVanillaOptions': False}

In [358]:
info = client.get_isolated_margin_account()
for i in range(len(info['assets'])):
    if client.get_isolated_margin_account()['assets'][i]['baseAsset']['asset'] == 'BTC' and client.get_isolated_margin_account()['assets'][i]['quoteAsset']['asset'] == 'USDT':
        print(client.get_isolated_margin_account()['assets'][1]['quoteAsset'])
        x = float(client.get_isolated_margin_account()['assets'][1]['baseAsset']['free'])
        print(x)

{'asset': 'USDT', 'borrowEnabled': True, 'borrowed': '0.00187876', 'free': '44.81296529', 'interest': '0', 'locked': '0', 'netAsset': '44.81108653', 'netAssetOfBtc': '0.00144874', 'repayEnabled': True, 'totalAsset': '44.81296529'}
6.89e-06


In [374]:
43.983698 - 44

-0.016302000000003147

In [14]:
free = float(client.get_isolated_margin_account(symbols='BTCTUSD')['assets'][0]['quoteAsset']['free'])
free
# 44.81296529 

43.983698

In [314]:
pd.DataFrame([info])

,asset,borrowEnabled,borrowed,free,interest,locked,netAsset,netAssetOfBtc,repayEnabled,totalAsset
0,BNB,True,0,0.00082516,0,0,0.00082516,0.00000654,True,0.00082516


In [398]:
client.create_margin_loan(symbol='BTCTUSD',asset='TUSD', amount=f'{155 - 43.97577754}',isIsolated='TRUE')

{'tranId': 139840937372, 'clientTag': ''}

In [400]:
client.repay_margin_loan(symbol='BTCTUSD',asset='TUSD', amount=f'{155 - 43.97577754}',isIsolated='TRUE')

{'tranId': 139840970090, 'clientTag': ''}

In [262]:
client.get_account_api_permissions()

{'ipRestrict': True,
 'createTime': 1688443776000,
 'enableSpotAndMarginTrading': True,
 'enableWithdrawals': False,
 'enableInternalTransfer': False,
 'enableMargin': True,
 'enableFutures': True,
 'permitsUniversalTransfer': True,
 'enableVanillaOptions': False,
 'enableReading': True}

In [20]:
# client.create_margin_loan(symbol='BTCUSDT',asset='USDT', amount='310',isIsolated='TRUE')
buy_order = client.create_margin_order(
    symbol='BTCTUSD',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    isIsolated='TRUE',
    quantity=0.0009)

In [29]:
buy_order

{'symbol': 'BTCTUSD',
 'orderId': 1704166497,
 'clientOrderId': 'U1p2phyamxFV7zaoaEk4j4',
 'transactTime': 1688513648632,
 'price': '0',
 'origQty': '0.0009',
 'executedQty': '0.0009',
 'cummulativeQuoteQty': '27.728721',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '30809.69',
   'qty': '0.0009',
   'commission': '0',
   'commissionAsset': 'BNB',
   'tradeId': 231541668}],
 'isIsolated': True,
 'selfTradePreventionMode': 'NONE'}

In [21]:
sell_order = client.create_margin_order(
    symbol='BTCTUSD',
    side=Client.SIDE_SELL,
    type=Client.ORDER_TYPE_MARKET,
    isIsolated='TRUE',
    quantity=0.0009)
client.repay_margin_loan(symbol='BTCUSDT',asset='USDT', amount='310',isIsolated='TRUE')

In [30]:
sell_order

{'symbol': 'BTCTUSD',
 'orderId': 1704166599,
 'clientOrderId': '91GTqQ05S9QRgD37gNHIZp',
 'transactTime': 1688513650136,
 'price': '0',
 'origQty': '0.0009',
 'executedQty': '0.0009',
 'cummulativeQuoteQty': '27.728019',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '30808.91',
   'qty': '0.0009',
   'commission': '0',
   'commissionAsset': 'BNB',
   'tradeId': 231541672}],
 'isIsolated': True,
 'selfTradePreventionMode': 'NONE'}